In [ ]:
# DP implementation of factorial
function Base.factorial(n::Int, factorials::Vector{Int})
    if n == 1 || n == 0 
        return 1
    end 

    if factorials[n] != 0  
        return factorials[n]
    else 
        fact =  n * factorial(n - 1, factorials)
        factorials[n] = fact
        return fact
    end 
end

# returns kth lexicogrpahical order of the first n ints 
# k ε [1,n!]
function nth_lex_permutation(n::Int, k::Int)
    k -= 1
    factorials = zeros(Int, 26)
        
    numbers = collect(1:n)
    permutation = Vector{Int}()

    for i in 1:n
        fact = factorial(n - i, factorials)
        index = div(k, fact)
        k %= fact

        push!(permutation, numbers[index+1])
        splice!(numbers, index+1)

    end
    
    return permutation
end


# n = 5  # num duplicates 
# k = 45  #  kth lexicogrpahical order of the first n ints [1:n!]

# perm = nth_lex_permutation(n, k)
# println(perm)

In [11]:
using Plots

# Create a matrix
matrix = [1 2 1; 4 5 1; 7 1 9]

# Create a 3D plot
surface(matrix)


Int64[]

In [ ]:
function linear_circ_helper(src_chroms::Vector{SubString{String}}) 
    linear_chroms = Vector{String}() 
    circ_chroms = Vector{String}()
    linear_idx_to_genome_idx = Dict{Int, Int}() 
    circ_idx_to_genome_idx = Dict{Int, Int}() 

    genome_idx = 1
    linear_idx = 1
    circ_idx = 1
    
    for c in src_chroms
        if c[1] == '.' 
            push!(linear_chroms, c)
            linear_idx_to_genome_idx[linear_idx] = genome_idx
            linear_idx += 1
        else 
            push!(circ_chroms, c)
            circ_idx_to_genome_idx[circ_idx] = genome_idx
            circ_idx += 1
        end 
        genome_idx += 1
    end 

    return linear_chroms, circ_chroms, linear_idx_to_genome_idx, circ_idx_to_genome_idx
end 


# insert random dup gene into the source string at a random location 
function randomly_insert_gene(src_chroms::Vector{SubString{String}}, rand_dup_gene::Char, alone::Bool, linear::Bool)   
    if alone 
        rand_idx = rand(1:length(src_chroms))
        if linear
            rand_dup_gene = "." * rand_dup_gene * "."
        end 
        insert!(src_chroms, rand_idx, string(rand_dup_gene))
        src = join(src_chroms, ",")

    else 
        linear_chroms, circ_chroms, linear_idx_to_genome_idx, circ_idx_to_genome_idx = linear_circ_helper(src_chroms)
       
        if linear # find all the linear chroms and choose one randomly to insert into 
            if linear_chroms == Vector{String}()   # no linear chroms
                # choose random index of genome and insert new linear chrom 
                rand_dup_gene = "." * rand_dup_gene * "."
                rand_idx = rand(1:length(src_chroms)+1)
                insert!(src_chroms, rand_idx, string(rand_dup_gene))
                src = join(src_chroms, ",")

            else  # randomly insert into existing linear chrom 
                rand_idx_lin_chroms = rand(1:length(linear_chroms))
                rand_idx_genome = linear_idx_to_genome_idx[rand_idx_lin_chroms]
                lin_chrom = linear_chroms[rand_idx_lin_chroms] 

                rand_idx_chrom = rand(2:length(lin_chrom))
                updated_chrom = lin_chrom[1:rand_idx_chrom-1] * string(rand_dup_gene) * lin_chrom[rand_idx_chrom:end]
                
                src_chroms[rand_idx_genome] = updated_chrom
                src = join(src_chroms, ",")
            end 

        else  # find all the circular chroms and choose one randomly to insert it into 
            if circ_chroms == Vector{String}()   # no circ chroms
                # choose random index of genome and insert new circ chrom 
                rand_idx = rand(1:length(src_chroms)+1)
                insert!(src_chroms, rand_idx, string(rand_dup_gene))
                src = join(src_chroms, ",")

            else  # randomly insert into existing linear chrom 
                rand_idx_circ_chroms = rand(1:length(circ_chroms))
                rand_idx_genome = circ_idx_to_genome_idx[rand_idx_circ_chroms]
                circ_chrom = circ_chroms[rand_idx_circ_chroms] 

                rand_idx_chrom = rand(1:length(circ_chrom)+1)
                updated_chrom = circ_chrom[1:rand_idx_chrom-1] * string(rand_dup_gene) * circ_chrom[rand_idx_chrom:end]
                
                src_chroms[rand_idx_genome] = updated_chrom
                src = join(src_chroms, ",")
            end 
        end 
    end  

    return src
end 


function generate_source_genome_with_xdup(n::Int, dup_genes:: Vector{Char})
    src = generate_genome_str(n)
    src_chroms = split(src, ",")
    
    for rand_dup_gene in dup_genes
        alone = rand(Bool)
        linear = rand(Bool)

        src = randomly_insert_gene(src_chroms, rand_dup_gene, alone, linear)
    end 
 
    return src
end 


# generates target and source genomes that contain the first n letters and adds x duplicate letters 
# x >= 1
function generate_genomes_with_xdup(n::Int, x::Int)
    # target 
    genes = first_n_letters(n)
    dup_genes = Vector{Char}()
    
    for i in 1:x
        rand_dup_gene = genes[rand(1:length(genes))]
        rand_idx = rand(1:length(genes)+1)
        insert!(genes, rand_idx, rand_dup_gene)
        push!(dup_genes, rand_dup_gene)
    end 
    target = join(genes)

    # source 
    src = generate_source_genome_with_xdup(n, dup_genes)

    check_src_tar_balanced(src, target)

    return src, target
end 

generate_genomes_with_xdup(5, 2)